In [33]:
import math
import tkinter as tk
def pyshader(func, w, h):
    scr = bytearray((0, 0, 0) * w * h)
    for y in range(h):
        for x in range(w):
            p = (w * y + x) * 3
            scr[p:p + 3] = [max(min(int(c * 255), 255), 0)
            for c in func(x / w, y / h)]
            return bytes('P6\n%d %d\n255\n' % (w, h), 'ascii') + scr
        
# Ваш код здесь:
def func(x, y):
    return x, y, 0

label = tk.Label()
img = tk.PhotoImage(data=pyshader(func, 256, 256)).zoom(2, 2)
label.pack()
label.config(image=img)
tk.mainloop()

In [48]:
import math
import tkinter as tk


def pyshader(func, w, h):
    scr = bytearray((0, 0, 0) * w * h)
    for y in range(h):
        for x in range(w):
            p = (w * y + x) * 3
            scr[p:p + 3] = [
                max(min(int(c * 255), 255), 0)
                for c in func(x / w, y / h)
            ]
    return bytes('P6\n%d %d\n255\n' % (w, h), 'ascii') + scr


def func(x, y):
    cx_green, cy_green = 0.4, 0.4
    r_green = 0.4

    cx_red, cy_red = 0.5, 0.5
    r_red = 0.4

    d_red = math.hypot(x - cx_red, y - cy_red)
    d_green = math.hypot(x - cx_green, y - cy_green)

    intensity_red = max(0, 1 - (d_red / r_red)**2)
    intensity_green = max(0, 1 - (d_green / r_green)**2)

    return intensity_red, intensity_green, 0


label = tk.Label()
img = tk.PhotoImage(
    data=pyshader(func, 256, 256)
).zoom(2, 2)
label.pack()
label.config(image=img)
tk.mainloop()


In [51]:
import math
import tkinter as tk


def pyshader(func, w, h):
    scr = bytearray((0, 0, 0) * w * h)
    for y in range(h):
        for x in range(w):
            p = (w * y + x) * 3
            scr[p:p + 3] = [
                max(min(int(c * 255), 255), 0)
                for c in func(x / w, y / h)
            ]
    return bytes('P6\n%d %d\n255\n' % (w, h), 'ascii') + scr


def func(x, y):
    cx, cy = 0.5, 0.5
    r = 0.4
    dx, dy = x - cx, y - cy
    dist2 = dx * dx + dy * dy

    if dist2 > r * r:
        return (0, 0, 0)

    # Рот
    angle = math.atan2(dy, dx)
    if -math.pi / 6 < angle < math.pi / 6:
        return (0, 0, 0)

    # Глаза
    ex, ey = 0.55, 0.3
    er = 0.05
    if (x - ex)**2 + (y - ey)**2 < er * er:
        return (0, 0, 0)

    # Тело
    return (1, 1, 0)


label = tk.Label()
img = tk.PhotoImage(
    data=pyshader(func, 256, 256)
).zoom(2, 2)
label.pack()
label.config(image=img)
tk.mainloop()


In [52]:
import math
import tkinter as tk
import hashlib


def pyshader(func, w, h):
    scr = bytearray((0, 0, 0) * w * h)
    for y in range(h):
        for x in range(w):
            p = (w * y + x) * 3
            scr[p:p + 3] = [
                max(min(int(c * 255), 255), 0)
                for c in func(x / w, y / h)
            ]
    return bytes('P6\n%d %d\n255\n' % (w, h), 'ascii') + scr


def noise(x, y):
    coord_str = f"{x:.5f}:{y:.5f}"

    # хэш
    h = hashlib.md5(coord_str.encode('utf-8')).hexdigest()

    val_32bit = int(h[:8], 16)
    return val_32bit / 0xFFFFFFFF


def func(x, y):
    n = noise(x, y)
    return n, n, n


label = tk.Label()
img = tk.PhotoImage(
    data=pyshader(func, 256, 256)
).zoom(2, 2)
label.pack()
label.config(image=img)
tk.mainloop()


In [53]:
import math
import tkinter as tk
import hashlib


def pyshader(func, w, h):
    scr = bytearray((0, 0, 0) * w * h)
    for y in range(h):
        for x in range(w):
            p = (w * y + x) * 3
            scr[p:p + 3] = [
                max(min(int(c * 255), 255), 0)
                for c in func(x / w, y / h)
            ]
    return bytes('P6\n%d %d\n255\n' % (w, h), 'ascii') + scr




def noise2d(i, j):
    """
    Псевдослучай для целочисленных (i, j).
    Возвращает число в [0..1].
    """
    s = f"{i}:{j}"
    h = hashlib.md5(s.encode()).hexdigest()
    return int(h[:8], 16) / 0xFFFFFFFF


def fade(t):
    """Классическая сглаживающая функция для value noise."""
    return t * t * t * (t * (t * 6 - 15) + 10)


def lerp(a, b, t):
    """Линейная интерполяция."""
    return a + (b - a) * t


def val_noise(x, y):
    """
    Интерполированный шум.
    1. Определяем целочисленные координаты (xi, yi).
    2. Берём четыре «шумовых» значения на углах клетки.
    3. Сглаживаем по x и y.
    """
    xi = int(math.floor(x))
    yi = int(math.floor(y))
    xf = x - xi
    yf = y - yi
    n00 = noise2d(xi,     yi)
    n10 = noise2d(xi + 1, yi)
    n01 = noise2d(xi,     yi + 1)
    n11 = noise2d(xi + 1, yi + 1)
    rx0 = lerp(n00, n10, fade(xf))
    rx1 = lerp(n01, n11, fade(xf))
    return lerp(rx0, rx1, fade(yf))


def func(x, y):
    """
    Для визуализации берём val_noise с некоторым масштабом.
    """
    scale = 8
    v = val_noise(scale * x, scale * y)
    return (v, v, v)


label = tk.Label()
img = tk.PhotoImage(
    data=pyshader(func, 256, 256)
).zoom(2, 2)
label.pack()
label.config(image=img)
tk.mainloop()


In [58]:
import math
import hashlib
import tkinter as tk


def pyshader(func, w, h):
    scr = bytearray((0, 0, 0) * w * h)
    for Y in range(h):
        for X in range(w):
            p = (w * Y + X) * 3
            col = func(X / w, Y / h)
            scr[p:p + 3] = [
                max(min(int(c * 255), 255), 0)
                for c in col
            ]
    return bytes(f'P6\n{w} {h}\n255\n', 'ascii') + scr


def noise2d(i, j):
    """
    Псевдослучай для целочисленных (i, j).
    Возвращает число в [0..1].
    """
    s = f"{i}:{j}"
    h = hashlib.md5(s.encode()).hexdigest()
    return int(h[:8], 16) / 0xFFFFFFFF


def fade(t):
    """Сглаживающая функция для value noise."""
    return t * t * t * (t * (t * 6 - 15) + 10)


def lerp(a, b, t):
    """Линейная интерполяция."""
    return a + (b - a) * t


def val_noise(x, y):
    """
    Обычный value noise c интерполяцией:
    1) Находим (xi, yi) и дробные (xf, yf).
    2) Берём шумы в четырёх углах клетки.
    3) Интерполируем по fade.
    """
    xi = int(math.floor(x))
    yi = int(math.floor(y))
    xf = x - xi
    yf = y - yi
    n00 = noise2d(xi,     yi)
    n10 = noise2d(xi + 1, yi)
    n01 = noise2d(xi,     yi + 1)
    n11 = noise2d(xi + 1, yi + 1)
    rx0 = lerp(n00, n10, fade(xf))
    rx1 = lerp(n01, n11, fade(xf))
    return lerp(rx0, rx1, fade(yf))


def fbm(x, y, octaves=5, lac=2.0, gain=0.5):
    """
    Функция fBm (фрактальный шум) на основе val_noise:
    - octaves: число октав
    - lac: во сколько раз увеличиваем частоту на каждую октаву
    - gain: во сколько раз уменьшаем амплитуду
    """
    value = 0.0
    amplitude = 1.0
    freq = 1.0
    sum_amp = 0.0
    for _ in range(octaves):
        value += val_noise(x * freq, y * freq) * amplitude
        sum_amp += amplitude
        freq *= lac
        amplitude *= gain
    # Нормируем сумму, чтобы оставаться в [0..1]
    return value / sum_amp


def func(x, y):
    # Увеличим масштаб, чтобы «облака» были покрупнее
    scale = 4.0
    # Вычислим фрактальный шум в [0..1]
    v = fbm(x * scale, y * scale, octaves=5)

    r = 1.0 - 0.8 * v
    g = 1.0 - 0.8 * v
    b = 1.0
    return (r, g, b)


label = tk.Label()
img = tk.PhotoImage(data=pyshader(func, 256, 256)).zoom(2, 2)
label.pack()
label.config(image=img)
tk.mainloop()
